<a href="https://colab.research.google.com/github/iamsoroush/mnist_inception_finetune/blob/master/mnist_xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Input, UpSampling3D
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.applications.xception import Xception

from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = (x_train / 255).astype('float32')
x_test = (x_test / 255).astype('float32')

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

y_train = to_categorical(y_train)

random_seed = 2
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=random_seed)

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
                             samplewise_center=False,  # set each sample mean to 0
                             featurewise_std_normalization=False,  # divide inputs by std of the dataset
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening
                             rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.1, # Randomly zoom image 
                             width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                             horizontal_flip=False,  # randomly flip images
                             vertical_flip=False)  # randomly flip images)

datagen.fit(x_train)

In [0]:
# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False)


input_xception = Input(shape=(28, 28, 1), dtype='float32', name='xception_input')

x = UpSampling3D(size=(3, 3, 3), data_format="channels_last")(input_xception)
x = base_model(x)

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# and a logistic layer
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
xception_model = Model(inputs=input_xception, outputs=predictions)

xception_model.summary()

83689472/83683744 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (None, 84, 84, 3)         0         
_________________________________________________________________
xception (Model)             multiple                  20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (De

In [0]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_1
13 block2_pool
14 batch_normalization_1
15 add_1
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_2
23 block3_pool
24 batch_normalization_2
25 add_2
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_3
33 block4_pool
34 batch_normalization_3
35 add_3
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_4
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_sepcon

In [0]:
for layer in base_model.layers[:36]:
   layer.trainable = False
    
xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (None, 84, 84, 3)         0         
_________________________________________________________________
xception (Model)             multiple                  20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total para

In [0]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.1, 
                                            min_lr=0.00001)

xception_model.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
epochs = 60
batch_size = 64
xception_history = xception_model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                                steps_per_epoch=len(x_train) / batch_size, epochs=epochs,
                                                validation_data=(x_val, y_val),
                                                callbacks=[learning_rate_reduction])

Epoch 1/60
750/750 [==============================] - 203s 270ms/step - loss: 0.4324 - acc: 0.8665 - val_loss: 0.3587 - val_acc: 0.9637
Epoch 2/60
750/750 [==============================] - 195s 260ms/step - loss: 0.1116 - acc: 0.9761 - val_loss: 0.0551 - val_acc: 0.9886
Epoch 3/60
750/750 [==============================] - 194s 259ms/step - loss: 0.0970 - acc: 0.9809 - val_loss: 0.0744 - val_acc: 0.9867
Epoch 4/60
750/750 [==============================] - 195s 259ms/step - loss: 0.0877 - acc: 0.9847 - val_loss: 0.0922 - val_acc: 0.9868
Epoch 5/60
750/750 [==============================] - 195s 259ms/step - loss: 0.0796 - acc: 0.9857 - val_loss: 0.2410 - val_acc: 0.9779
Epoch 6/60
750/750 [==============================] - 194s 259ms/step - loss: 0.0843 - acc: 0.9866 - val_loss: 0.1881 - val_acc: 0.9774
Epoch 7/60
750/750 [==============================] - 194s 259ms/step - loss: 0.0780 - acc: 0.9874 - val_loss: 0.0614 - val_acc: 0.9894

Epoch 00007: ReduceLROnPlateau reducing learnin